In [1]:
%pip install pandas

In [2]:
import pandas as pd

df = pd.read_csv('connection_graph.csv')

df.head()

In [10]:
import pandas as pd
from math import inf
import heapq
import time
import sys

all_stations = pd.concat([df["start_stop"], df["end_stop"]])
unique_stations = all_stations.unique()
unique_stations.sort()

def get_time_in_seconds(time_str: str):
    hours, minutes, seconds = time_str.split(":")
    return int(hours) * 3600 + int(minutes) * 60 + int(seconds)

def get_time_difference(departure: int, arrive: int):
    if departure > arrive:
        arrive += 3600 * 24
    return arrive - departure

def format_time(seconds: int): 
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours:02}:{minutes:02}:{secs:02}"

def when_to_ride(station_a, station_b, criteria, start_time):
    nodes = {station: inf for station in unique_stations}
    start_seconds = get_time_in_seconds(start_time)
    nodes[station_a] = start_seconds
    print(f"Starting at station {station_a} at {start_time} ({nodes[station_a]} seconds)")
    
    prev = {}  # Maps station -> (previous_station, (line, board_stop, board_time, alight_stop, alight_time))
    visited = set()
    pq = []
    heapq.heappush(pq, (nodes[station_a], station_a))
    
    while pq:
        current_time, current_node = heapq.heappop(pq)
        if current_time > nodes[current_node]:
            continue
        
        # print(f"Checking node {current_node} at time {format_time(current_time)}")
        if current_node == station_b:
            path = []
            node = station_b
            while node in prev:
                ride = prev[node][1]
                path.append(ride)
                node = prev[node][0]
            path.reverse()
            print("Found final node")
            return current_time, path
        
        visited.add(current_node)
        neighbors = df[df['start_stop'] == current_node]
        # print(f"Found {len(neighbors)} neighbors for {current_node}")
        
        for idx, row in neighbors.iterrows():
            neighbor = row["end_stop"]
            if neighbor in visited:
                continue
            
            sched_departure = get_time_in_seconds(row["departure_time"])
            current_day_time = current_time % 86400
            day_start = current_time - current_day_time
            if sched_departure < current_day_time:
                abs_departure = day_start + sched_departure + 86400
            else:
                abs_departure = day_start + sched_departure
            
            ride_duration = get_time_difference(
                get_time_in_seconds(row["departure_time"]),
                get_time_in_seconds(row["arrival_time"])
            )
            new_time = abs_departure + ride_duration
            # print(f"From {current_node} to {neighbor}: line {row['line']}, scheduled depart {row['departure_time']}, arrive {row['arrival_time']}; abs depart {format_time(abs_departure)}, ride duration {ride_duration} -> new time {format_time(new_time)}")
            
            if new_time < nodes[neighbor]:
                nodes[neighbor] = new_time
                prev[neighbor] = (current_node, (row["line"], current_node, abs_departure, neighbor, new_time))
                heapq.heappush(pq, (new_time, neighbor))
    
    print("No route found")
    return inf, []

start_comp = time.time()
result_time, path = when_to_ride('Śliczna', 'most Grunwaldzki', 't', "08:50:00")
end_comp = time.time()

if result_time == inf:
    print("No route found")
else:
    print("\nTravel Schedule:")
    current_line = None
    segment_start_station = None
    segment_start_time = None
    previous_ride_end_station = None
    previous_ride_end_time = None

    for ride in path:
        line = ride[0]
        if line != current_line:
            # print previous line segment if any
            if current_line is not None:
                print(f"Linia {current_line}, wsiadam: {segment_start_time} {segment_start_station}, wysiadam: {previous_ride_end_time} {previous_ride_end_station}")
            current_line = line
            segment_start_station = ride[1]
            segment_start_time = format_time(ride[2])
        previous_ride_end_station = ride[3]
        previous_ride_end_time = format_time(ride[4])

    # print the final segment
    if current_line is not None:
        print(f"Linia {current_line}, wsiadam: {segment_start_time} {segment_start_station}, wysiadam: {previous_ride_end_time} {previous_ride_end_station}")

    print("\nEarliest arrival time:", format_time(result_time))

comp_time = end_comp - start_comp
sys.stderr.write(f"Cost: {result_time} seconds; Computation time: {comp_time:.4f} seconds\n")

Starting at station Śliczna at 08:50:00 (31800 seconds)
Found final node

Travel Schedule:
Linia 612, wsiadam: 08:50:00 Śliczna, wysiadam: 08:54:00 DWORZEC AUTOBUSOWY
Linia K, wsiadam: 08:54:00 DWORZEC AUTOBUSOWY, wysiadam: 08:57:00 DWORZEC GŁÓWNY
Linia 4, wsiadam: 08:57:00 DWORZEC GŁÓWNY, wysiadam: 09:08:00 most Grunwaldzki

Earliest arrival time: 09:08:00


Cost: 32880 seconds; Computation time: 12.5378 seconds


55